# Firms population


In [1]:
#Load modules

import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from numpy import log10
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Load data
### Here we load the data aggregated at (firm, year, flow) level. The only variable we know for each (firm, year, flow) is the 'VART_sum', that is, the sum of transaction values at border.

In [2]:

path1 = './../../../../../media/miglesia/Elements/export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

firm_mx_y = pd.read_csv(path1+'/samplings/full_firm_mx_yr.csv')

In [3]:
#Sample of the loaded dataset

ix = ['ID', 'IMPORT', 'YEAR']
display(firm_mx_y.sort_values(by = ix).head(20).set_index(ix))
print '...'

VART_sum
ID IMPORT YEAR          
0  0      1997  39913032
          1998  25985808
          1999  44626344
          2000  15188499
          2001  60013092
          2002  16027572
          2003  10909484
          2004  17354863
          2005   8502114
          2006  14040307
          2007  14002190
          2008  51185304
          2009   9799932
          2010  97361175
          2011   1435637
          2012    131222
          2013      1109
   1      1997  46236989
          1998  29903677
          1999  51071786

...


### Join Firms name

In [4]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 200

#Firms data (name, activity, employees)

firms = pd.read_csv('./../data/sirc-50225_50225_71171_201805_L_P_20180624_011932315.csv', delimiter = ';')
firms = firms[['SIREN','L1_NORMALISEE','LIBTEFET','APET700','LIBAPET']].set_index('SIREN')
firms.index.name = 'ID'

In [5]:
path1 = './../../../../../media/miglesia/Elements/export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

df = pd.read_csv(path1+'/firm_lev_sample.csv')

# 4 year period
df['PERIOD'] = (df.YEAR - 1997) % 4

n = df[['YEAR', 'PERIOD']].drop_duplicates().groupby('PERIOD').count().reset_index()
n.columns = ['PERIOD', 'n']

# drop year 1997, so that we got 4 periods of four years

df = df.loc[df.YEAR != 1997]

# Information on Combined nomenclature product codes (CN) and CPA activity codes
CN_full = pd.read_csv('data/CN_full.csv')
CPA_full = pd.read_csv('data/CPA_full.csv')
df = df.merge(CN_full[['CN ID 8', 'CN ID 4', 'CN label 4']])

df.sample(5)

IOError: File data/CN_full.csv does not exist

In [ ]:

dataset = df.loc[df.PERIOD == 3]

df_ = dataset.groupby(['ID', 'IMPORT']).agg({'VART': ['count','sum']}).sort_values(by = ('VART', 'sum'), ascending = False)
largest = df_.astype(float).merge(firms, left_index = True, right_index = True)
largest.head(20)

In [ ]:
row_list = []
for ix in largest.head(100).index:
    dfv = dataset.loc[(dataset.ID == ix[0]) &(dataset.IMPORT == ix[1]) ].groupby('CN label 4').sum()[['VART']].sort_values(by = 'VART', ascending = False)
    dfx = (100*dfv/dfv.sum()).round()
    dfxv_p = pd.concat([dfv, dfx], axis = 1)
    dfxv_p.columns = ['VART', 'VART_pct']
    
    dfv = dataset.loc[(dataset.ID == ix[0]) &(dataset.IMPORT == ix[1]) ].groupby('PYOD').sum()[['VART']].sort_values(by = 'VART', ascending = False)
    dfx = (100*dfv/dfv.sum()).round()
    dfxv_c = pd.concat([dfv, dfx], axis = 1)
    dfxv_c.columns = ['VART', 'VART_pct']

    try: 
        row_list += [pd.DataFrame(pd.concat([pd.DataFrame({'ID':[ ix[0]], 'IMPORT': [ix[1]]}), 
                                            dfxv_p[['VART_pct']].reset_index().iloc[[0]],
                                            dfxv_c[['VART_pct']].reset_index().iloc[[0]]], axis = 1)).set_index('ID')]
    except:
        pass
    
firm_info = pd.concat(row_list)
firm_info.columns = ['IMPORT','main CN4', 'main CN4 pct','main PYOD', 'main PYOD pct']
firm_info = firm_info.reset_index().set_index(['ID', 'IMPORT'])

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

# pd.set_option('display.float_format', '{:.2g}'.format)

res = df_.merge(firm_info, left_index=  True, right_index = True).sort_values(by = ('VART', 'sum'), ascending = False)
res = res.reset_index().set_index('ID').merge(firms, left_index = True, right_index = True).sort_values(by = ('VART', 'sum'), ascending = False)
res[('VART', 'sum')] = res[('VART', 'sum')].astype(float)
res


In [ ]:
import webbrowser

for siren in df_.index[30:60]:
    # google siren code
    webbrowser.open('https://www.google.com/search?q='+str(siren)+'+siren+score3')



## Other stuff

In [ ]:
grouped = df2012_sam_1.loc[df2012_sam_1.ID == 542034921].groupby('CN ID 8')

df_1 = grouped.sum()[['VFTE', 'VART']].astype(float)#.style.bar(color='#d65f5f')

df_2 = grouped.nunique()[['MONTH', 'DEPT', 'PYOD', 'CPA6','PAYP']]#.style.bar(color='#d65f5f')

pd.concat([df_1, df_2], axis = 1).sort_values(by = 'VART', ascending = False).style.bar(color='#d65f5f')

In [ ]:
df = df2012_sam_1

plt.hist(np.log10(df[13].values), 50)
plt.show()

In [ ]:
df_95['log_bin'] = np.log10(df_95[13]).round(2)
df_95.groupby('log_bin').sum()[13].plot(marker = 'o')

In [ ]:
df_95.loc[df_95[2].isin([1, 3]), 13].sum()
df_95.loc[df_95[2].isin([2, 4]), 13].sum()

In [ ]:

print true_divide(df_9999.loc[:, 12:].sum(), df_9.loc[:, 12:].sum())
print true_divide(df_999.loc[:, 12:].sum(), df_9.loc[:, 12:].sum())
print true_divide(df_99.loc[:, 12:].sum(), df_9.loc[:, 12:].sum())

In [ ]:
# df = pd.read_csv(path1+'/DP1610_MAASTRICHT1_2012.txt', delimiter = ';', header = None, nrows = 3000000)
# d = dict(df.dtypes)

# df = pd.read_csv(path1+'/DP1610_MAASTRICHT1_2012.txt', delimiter = ';', header = None,
#                 usecols=range(14), skiprows=lambda i: i % n != 0, low_memory=True, dtype = d)
df.columns = [u'YEAR', u'MONTH', u'FLUX', u'ID', u'DEPT', u'CN ID 8', u'CPA6',
       u'PYOD', u'PAYP', u'VAT', u'PRIFAC', u'DEVFAC', u'VFTE', u'VART']

print len(df)
df1_2012 = reduce_mem(df)

print(mem_usage(df))
print(mem_usage(df1_2012))

df1_2012

In [ ]:
# drafts from now on

In [ ]:
df1_2012 = pd.read_csv('./data/type1/DP1610_MAASTRICHT1_2012.txt', delimiter = ';', header = None)
pd.read_csv('./data/type1/DP1610_MAASTRICHT1_2013.txt', delimiter = ';', header = None)

In [ ]:
df2_2012 = reduce_mem(pd.read_csv('./data/type2/DP1611_MAASTRICHT2_2012.txt', delimiter = ';', header = None))
print(mem_usage(df2_2012_o))



# df2_2013 = pd.read_csv('./data/type2/DP1611_MAASTRICHT2_2013.txt', delimiter = ';', header = None)

In [ ]:
l = [df2_2012, df2_2013]

In [ ]:
for df in l:
    print len(df)

In [ ]:


gl = df2_2013

gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(gl))
print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
for df in l:
    display(df.sample(10))
    print df.max()